Imports

In [1]:
from math import sqrt
from numpy import concatenate
from matplotlib import pyplot as plt
%matplotlib inline
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
import numpy as np
import pandas as pd
import time

Using Theano backend.


In [2]:
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences

## Baseline LSTM + room to change network architecture

In [53]:
#df = pd.read_csv('pollution.csv', header=0, index_col=0)

#dataset = pd.read_csv('beijing_weather_2015_2017.csv', header=0, index_col=0)

df = pd.read_csv('merged_final_v2.csv', header=0, index_col=0)
#df['cumulative_snow_hours'].fillna(0, inplace=True)
# Optionally drop wind dir
df.columns


//anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Index(['air_pressure', 'air_temp', 'cumulative_rain_hours',
       'cumulative_snow_hours', 'date', 'day', 'dew_point', 'hour', 'month',
       'pm2.5', 'wind_dir', 'wind_speed', 'year'],
      dtype='object')

In [54]:
#target variable
y = df['pm2.5']

#train, test = train_test_split(df, test_size=.20, random_state=789)
X_train, X_test, y_train, y_test = train_test_split(df, y ,test_size=0.2)

In [55]:
len(X_train)

55176

In [56]:
len(X_test)

13794

In [69]:
def convert_to_list(df):
    df_list = df.values.tolist()
    return df_list

def normalize_windows(window_data):
    normalized_data = []
    for window in window_data:
        normalized_window = [((float(p) / float(window[0])) - 1) for p in window]
        normalized_data.append(normalized_window)
    return normalized_data

def lists_to_sequences(list1, seq_len, normalize=True):
    sequences = []
    for index in range(len(list1) - seq_len):
        sequences += [list1[index: index + seq_len]]
    if normalize:
        sequences = normalize_windows(sequences) 
    return np.array(sequences)

In [58]:
X_train_list = convert_to_list(X_train)
X_test_list = convert_to_list(X_test)
y_train_list = convert_to_list(y_train)
y_test_list = convert_to_list(y_test)

num_days = 2
num_hours = num_days * 24
sequences = lists_to_sequences(X_train_list, num_hours)
print(len(sequences[1]))


48


In [60]:
print(X_train.shape[1])

13


In [62]:
X_train_list = np.reshape(X_train_list, (X_train.shape[0], X_train.shape[1], 1))
X_test_list = np.reshape(X_test_list, (X_test.shape[0], X_test.shape[1], 1))  

In [63]:
X_train.head()

,air_pressure,air_temp,cumulative_rain_hours,cumulative_snow_hours,date,day,dew_point,hour,month,pm2.5,wind_dir,wind_speed,year
23484,1017.000000,26.0,0.0,0.0,2012-09-05,5,12,12,9,31.000000,SE,1.79,2012
10379,1030.000000,8.0,0.0,0.0,2011-03-09,9,-16,11,3,19.000000,NW,6.71,2011
8215,1014.000000,-3.0,0.0,0.0,2010-12-09,9,-8,7,12,155.000000,SE,1.79,2010
50977,10165.197636,140.0,10.0,82.0,2015-10-26,26,60,7,10,29.514286,220.0,30.00,2015
17189,1034.000000,-5.0,0.0,0.0,2011-12-18,18,-18,5,12,13.000000,NE,8.94,2011


In [64]:
from keras.models import Model, Sequential
from keras.layers import LSTM, Dense, TimeDistributed, Dropout, Activation



def build_model(layers):
    model = Sequential()

    model.add(LSTM(
        input_shape=(layers[1], layers[0]),
        output_dim=layers[1],
        return_sequences=True))
    model.add(Dropout(0.2))

    model.add(LSTM(
        layers[2],
        return_sequences=False))
    model.add(Dropout(0.2))

    model.add(Dense(
        output_dim=layers[3]))
    model.add(Activation("linear"))

    start = time.time()
    model.compile(loss="mse", optimizer="rmsprop")
    print("> Compilation Time : ", time.time() - start)
    return model

In [65]:
def predict_point_by_point(model, data):
    #Predict each timestep given the last sequence of true data, in effect only predicting 1 step ahead each time
    predicted = model.predict(data)
    predicted = np.reshape(predicted, (predicted.size,))
    return predicted

def predict_sequence_full(model, data, window_size):
    #Shift the window by 1 new prediction each time, re-run predictions on new window
    curr_frame = data[0]
    predicted = []
    for i in range(len(data)):
        predicted.append(model.predict(curr_frame[newaxis,:,:])[0,0])
        curr_frame = curr_frame[1:]
        curr_frame = np.insert(curr_frame, [window_size-1], predicted[-1], axis=0)
    return predicted

def predict_sequences_multiple(model, data, window_size, prediction_len):
    #Predict sequence of 50 steps before shifting prediction run forward by 50 steps
    prediction_seqs = []
    for i in range(int(len(data)/prediction_len)):
        curr_frame = data[i*prediction_len]
        predicted = []
        for j in range(prediction_len):
            predicted.append(model.predict(curr_frame[newaxis,:,:])[0,0])
            curr_frame = curr_frame[1:]
            curr_frame = np.insert(curr_frame, [window_size-1], predicted[-1], axis=0)
        prediction_seqs.append(predicted)
    return prediction_seqs

In [66]:
#plotting functions

def plot_results(predicted_data, true_data):
    fig = plt.figure(facecolor='white')
    ax = fig.add_subplot(111)
    ax.plot(true_data, label='True Data')
    plt.plot(predicted_data, label='Prediction')
    plt.legend()
    plt.show()

def plot_results_multiple(predicted_data, true_data, prediction_len):
    fig = plt.figure(facecolor='white')
    ax = fig.add_subplot(111)
    ax.plot(true_data, label='True Data')
    #Pad the list of predictions to shift it in the graph to it's correct start
    for i, data in enumerate(predicted_data):
        padding = [None for p in range(i * prediction_len)]
        plt.plot(padding + data, label='Prediction')
        plt.legend()
    plt.show()

In [76]:
def run_all(X_train, y_train, X_test, y_test, epochs, batch, seq_len):
    global_start_time = time.time()

    model = build_model([1, 50, 100, 1])
    
    model.fit(
        X_train,
        y_train,
        batch_size=batch,
        nb_epoch=epochs,
        validation_split=0.05)

    """
    predictions = predict_sequences_multiple(model, X_test, seq_len, 50)
    #predicted = lstm.predict_sequence_full(model, X_test, seq_len)
    #predicted = lstm.predict_point_by_point(model, X_test)        

    print('Training duration (s) : ', time.time() - global_start_time)
    plot_results_multiple(predictions, y_test, 50)
    return predictions
    """


run_all(X_train_list, y_train_list, X_test_list, y_test_list, epochs=1, batch=512, seq_len=48)

> Compilation Time :  0.00721287727355957


Exception: Error when checking model input: expected lstm_input_8 to have shape (None, 50, 1) but got array with shape (55176, 13, 1)

In [ ]:
#doing it across batch size
batch_size_list = range(10,100,10)
predictions_list = []
for bs in batch_size_list:
    predictions = run_all(X_train, y_train, X_test, y_test, epochs=1, batch_size=bs, seq_len=48)
    predictions_list += [predictions,]